<a href="https://colab.research.google.com/github/DangoPlus/utils/blob/main/user_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
!pip install openai pandas scikit-learn pydantic

In [118]:
from openai import OpenAI
import json
from IPython.display import display, HTML
from sklearn.metrics import precision_score, recall_score, f1_score
from concurrent.futures import ThreadPoolExecutor, as_completed
import csv
import pandas as pd
import os
from pydantic import BaseModel

In [119]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [120]:
client = OpenAI()

class Analysis(BaseModel):
    review: str
    first: str
    second: str
    third: str
    emotion: str


In [121]:
# 替换这里的路径为你的CSV文件路径
file_path = '/content/drive/MyDrive/misc/data6.csv'

# 读取CSV文件
df = pd.read_csv(file_path)

In [122]:
angleprompt = '''
You are a data analyst. You analyze the input content from users and give structured analysis results.
[{"一级维度":"食品安全","二级维度":"新鲜度","三级维度":"发酸，发臭，腥味，不新鲜等"},{"一级维度":"食品安全","二级维度":"异物感","三级维度":"头发，昆虫，塑料，纸巾等"},{"一级维度":"食品安全","二级维度":"不良反应","三级维度":"腹泻，胃痛，呕吐，头晕等"},{"一级维度":"产品口味&外观","二级维度":"口味适应","三级维度":"咸，淡，甜腻，难吃等"},{"一级维度":"产品口味&外观","二级维度":"口感体验","三级维度":"偏干硬，偏烫，夹生不熟等"},{"一级维度":"产品口味&外观","二级维度":"餐食份量","三级维度":"分量过少，分量不符等"},{"一级维度":"产品口味&外观","二级维度":"美观度","三级维度":"颜色奇怪"},{"一级维度":"餐厅服务","二级维度":"出餐速度","三级维度":""},{"一级维度":"餐厅服务","二级维度":"出餐错漏","三级维度":""},{"一级维度":"餐厅服务","二级维度":"餐具调料","三级维度":""},{"一级维度":"餐厅服务","二级维度":"业务熟练度","三级维度":""},{"一级维度":"餐厅服务","二级维度":"店员收餐","三级维度":""},{"一级维度":"餐厅服务","二级维度":"店员响应","三级维度":""},{"一级维度":"餐厅服务","二级维度":"店员散漫","三级维度":""},{"一级维度":"餐厅服务","二级维度":"等位秩序","三级维度":""},{"一级维度":"用餐环境","二级维度":"桌面清洁","三级维度":""},{"一级维度":"用餐环境","二级维度":"地板清洁","三级维度":""},{"一级维度":"用餐环境","二级维度":"座椅间距","三级维度":""},{"一级维度":"用餐环境","二级维度":"门店空间","三级维度":""},{"一级维度":"用餐环境","二级维度":"座椅舒适","三级维度":""},{"一级维度":"用餐环境","二级维度":"用餐氛围","三级维度":""},{"一级维度":"外卖配送","二级维度":"配送速度","三级维度":""},{"一级维度":"外卖配送","二级维度":"配送地址","三级维度":"送错"},{"一级维度":"外卖配送","二级维度":"配送态度","三级维度":""},{"一级维度":"外卖配送","二级维度":"外卖包装","三级维度":"包装不严密、漏洒、保温失效"},{"一级维度":"外卖配送","二级维度":"用户要求","三级维度":""},{"一级维度":"其他反馈","二级维度":"取消订单","三级维度":""},{"一级维度":"其他反馈","二级维度":"费用退款","三级维度":""},{"一级维度":"其他反馈","二级维度":"其他","三级维度":""}]
You need to analyze the user's content based on the above dimensions.first Put the user's input into the review. then Find out which primary dimension the content belongs to, then determine the secondary dimension within that primary dimension, and finally identify the tertiary dimension within the secondary dimension. Place the matched primary dimension in "first," the secondary dimension in "second," and the tertiary dimension in "third." When outputting the third-level dimension, only output the most relevant "one" separated by commas.Strictly use only the descriptive words that exist in the dimensions. Also, analyze the emotion of the user's input content, marking it as "Positive" or "Negative" and place this in "emotion."
'''

In [123]:
def handleAnalysis(input):
  completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": angleprompt},
        {"role": "user", "content": input},
    ],
    response_format=Analysis,
)
  # print(completion.choices[0].message.parsed)
  return completion.choices[0].message.parsed

In [124]:
# 假设我们处理数据的函数是这样的，这里以一个简单的示例为例
def process_data(row):
    # 假设我们的处理只是将某个字段加1
    # result = row['your_column_name'] + 1
    result = handleAnalysis(row)
    return result

In [125]:
# 结果列表
results = []

# 遍历DataFrame的每一行
# for index, row in df.iloc[:2].iterrows():
for index, row in df.iterrows():
    # 应用函数
    result = process_data(row['review'])
    # 将结果添加到列表中
    results.append(result)
# for item in results[:5]:
#     print(item)
# 如果你想要将结果保存到新的CSV文件
results_df = pd.DataFrame(results, columns=['review', 'first', 'second', 'third', 'emotion'])
for column in results_df.columns:
    results_df[column] = results_df[column].apply(lambda x: x[1])
# results_df.to_csv('processed_results.csv', index=False)# 打印表格
display(results_df)

,review,first,second,third,emotion
0,这个饮料坏的平时不是这个味道的今天一股酸带点臭的味道，变质XXXXXXXXXXX,食品安全,新鲜度,发酸，发臭,Negative
1,客户表示X英寸芝香烤肠卷边美国风情土豆培根比萨吃着吃着发现里面有个小虫,食品安全,异物感,昆虫,Negative
2,X英寸轻巧薄脆经典意式肉酱比萨OC有图披萨上有头发请核实处理XXXXXXXXXXX,食品安全,异物感,头发,Negative
3,顾客表示收到的新意大利肉酱面有塑料，需要回电,食品安全,异物感,塑料,Negative
4,XX英寸肉肉香粉松芝士卷边美国风情土有虫子，回电,食品安全,异物感,昆虫,Negative
5,X英寸双厚蒜香黄油风味照烧风味牛肉吃出头发,食品安全,异物感,头发,Negative
6,X英寸轻巧薄脆摩洛哥风情浓溢鸡腿肉比萨有类似塑料的异物,食品安全,异物感,塑料,Negative
7,X英寸双层芝心卷边金沙咸蛋黄嫩鸡比萨有毛发,食品安全,异物感,头发,Negative
8,X英寸咸蛋黄风味流沙卷边金沙咸蛋黄嫩鸡比萨上有疑似头发丝的东西，需回电XXXXXXXXXXX,食品安全,异物感,头发,Negative
9,XX英寸芝香烤肠卷边照烧风味牛有一根头发麻烦核实XXXXXXXXXXX,食品安全,异物感,头发,Negative


In [126]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=results_df)

https://docs.google.com/spreadsheets/d/1c4QkOCHGZE64eWokUwMnNFTXHQTQttpWAKmxaduLkqg#gid=0


/usr/local/lib/python3.10/dist-packages/google/colab/sheets.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return frame.applymap(_clean_val).replace({np.nan: None})
